In [1]:
from conllu import parse

In [3]:
with open('data/hungarian.conllu', 'r', encoding='utf8') as file:
    data = file.read()

sentences = parse(data)

In [6]:
for token in sentences[0]:
    print(token)

A
macska
a
szekrényben
van
.


In [11]:
sentences[0]

TokenList<A, macska, a, szekrényben, van, ., metadata={generator: "UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe", udpipe_model: "hungarian-szeged-ud-2.12-230717", udpipe_model_licence: "CC BY-NC-SA", newdoc: "None", newpar: "None", sent_id: "1", text: "A macska a szekrényben van."}>

Exploring Hungarian data

In [23]:
import spacy
from spacy.tokens import Doc, Token
from conllu import parse_incr
from conllu import parse
from spacy.vocab import Vocab
from spacy import displacy
from pprint import pprint

nlp = spacy.blank("hu")  
vocab = nlp.vocab

if not Token.has_extension("xpos"):
    Token.set_extension("xpos", default=None)
if not Token.has_extension("deps"):
    Token.set_extension("deps", default=None)
if not Token.has_extension("misc"):
    Token.set_extension("misc", default=None)


def process_conllu_file(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for conllu_sentence in parse_incr(file):
            doc = conllu_to_spacy_doc(conllu_sentence, vocab)
            docs.append(doc)
    return docs

def conllu_to_spacy_doc(conllu_sentence, vocab):
    words = [token["form"] for token in conllu_sentence]
    heads = [token["head"] if token["head"] is not None else 0 for token in conllu_sentence]  # Head = 0 para raíces
    deps = [token["deprel"].split(":")[0] for token in conllu_sentence]
    lemmas = [token["lemma"] for token in conllu_sentence]
    pos_tags = [token["upostag"] for token in conllu_sentence]
    xpos_tags = [token["xpos"] for token in conllu_sentence]
    morph_feats = [token["feats"] or {} for token in conllu_sentence]
    misc_info = [token["misc"] or {} for token in conllu_sentence]
    deps_info = [token["deps"] or {} for token in conllu_sentence]

    # Crear el objeto Doc
    doc = Doc(vocab, words=words)

    # Asignar los atributos a cada token
    for i, token in enumerate(doc):
        token.lemma_ = lemmas[i]
        
        # Asignar el Universal POS a `pos_`, o usar `tag_` si no es válido
        if pos_tags[i] != "_":
            token.pos_ = pos_tags[i]
        elif xpos_tags[i] is not None:  # Verificar si xpos_tags[i] no es None antes de asignarlo
            token.tag_ = xpos_tags[i]  # Usar `xpos` como `tag_` si `upos` es "_"

        token.dep_ = deps[i]
        
        # Verificar que heads[i] sea un número entero antes de asignarlo como cabeza
        if isinstance(heads[i], int) and heads[i] > 0:
            token.head = doc[heads[i] - 1]  # Usar head como índice (spaCy usa índice 0 para la raíz)
        else:
            token.head = token  # Si head es 0 o no válido, el token se apunta a sí mismo como raíz

        # Asignar xpos, deps y misc como atributos personalizados
        token._.xpos = xpos_tags[i]
        token._.deps = deps_info[i]
        token._.misc = misc_info[i]

        # Convertir `feats` a un formato que spaCy entiende y asignarlo a morph
        if morph_feats[i]:
            feats_str = "|".join(f"{k}={v}" for k, v in morph_feats[i].items())
            token.set_morph(feats_str)

    return doc

def render_doc_by_sent_id(docs, sentences, target_sent_id):
    # Buscar el índice del `sent_id` deseado en los metadatos de `sentences`
    for i, sentence in enumerate(sentences):
        if sentence.metadata.get("sent_id") == target_sent_id:
            # Renderizar el Doc correspondiente usando `displacy`
            displacy.render(docs[i], style="dep", jupyter=True)
            return  # Salimos después de encontrar y renderizar
    print(f"Oración con sent_id '{target_sent_id}' no encontrada.")


file_path = "data/hungarian.conllu"
docs = process_conllu_file(file_path)



In [36]:
render_doc_by_sent_id(docs, sentences, '1')
# The cat is in the wardrobe.

for token in sentences[0]:
    print(token.items())

dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '0:1'})])
dict_items([('id', 2), ('form', 'macska'), ('lemma', 'macska'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '2:8'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '9:10'})])
dict_items([('id', 4), ('form', 'szekrényben'), ('lemma', 'szekrény'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Ine', 'Number': 'Sing'}), ('head', 5), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '11:22'})])
dict_items([('id', 5), ('form', 'van'), ('lemma', 'van'), ('upos', 'VERB'), ('xpos', None), ('fe

In [27]:
pprint(sentences[0][3].items())

dict_items([('id', 4), ('form', 'szekrényben'), ('lemma', 'szekrény'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Ine', 'Number': 'Sing'}), ('head', 5), ('deprel', 'obl'), ('deps', None), ('misc', {'TokenRange': '11:22'})])


In [32]:
print(sentences[0][3].get('deprel'))
print(sentences[0][3].get('feats').get('Case'))

obl
Ine


In [ ]:
pprint(sentences[0][3].items())

Ine


In [ ]:
render_doc_by_sent_id(docs, sentences, '2')
#She enters the shop.

print(sentences[1])

for token in sentences[1]:
    print(token.items())

TokenList<Ő, bemegy, a, boltba, ., metadata={sent_id: "2", text: "Ő bemegy a boltba."}>
dict_items([('id', 1), ('form', 'Ő'), ('lemma', 'ő'), ('upos', 'PRON'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '28:29'})])
dict_items([('id', 2), ('form', 'bemegy'), ('lemma', 'bemegy'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Definite': 'Ind', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '30:36'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '37:38'})])
dict_items([('id', 4), ('form', 'boltba'), ('lemma', 'bolt'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom',

In [42]:
# boltba 'shop' is glossed as nominative.
sentences[1][3].items()

dict_items([('id', 4), ('form', 'boltba'), ('lemma', 'bolt'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '39:45'})])

In [51]:
render_doc_by_sent_id(docs, sentences, '7')

for token in sentences[6]:
    print(token.items())

dict_items([('id', 1), ('form', 'A'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 2), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '155:156'})])
dict_items([('id', 2), ('form', 'madár'), ('lemma', 'madár'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '157:162'})])
dict_items([('id', 3), ('form', 'a'), ('lemma', 'a'), ('upos', 'DET'), ('xpos', None), ('feats', {'Definite': 'Def', 'PronType': 'Art'}), ('head', 4), ('deprel', 'det'), ('deps', None), ('misc', {'TokenRange': '163:164'})])
dict_items([('id', 4), ('form', 'fánál'), ('lemma', 'fánál'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Case': 'Nom', 'Number': 'Sing'}), ('head', 5), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '165:170'})])
dict_items([('id', 5), ('form', 'pihen'), ('lemma', 'pihen'), ('upos', 'VERB'), ('xpos', Non

In [ ]:
sentences[6][3].get('feats').get('Case')

# The bird rests next to the tree.

#The output should say 'adessive'

'Nom'